In [1]:
import pandas as pd
import sqlite3
import random
import os
import hashlib

conn = sqlite3.connect('wikireddit.db')

In [2]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

for table in cursor.fetchall():
    cursor.execute(f"PRAGMA table_info({table[0]})")
    print(table)
    display(pd.DataFrame(cursor.fetchall()))

('posts',)


,0,1,2,3,4,5
0,0,subreddit_id,TEXT,0,None,0
1,1,crosspost_parent_id,TEXT,0,None,0
2,2,post_id,TEXT,0,None,0
3,3,created_at,TIMESTAMP,0,None,0
4,4,updated_at,TIMESTAMP,0,None,0
5,5,language_code,TEXT,0,None,0
6,6,score,INTEGER,0,None,0
7,7,upvote_ratio,REAL,0,None,0
8,8,gildings,INTEGER,0,None,0
9,9,num_comments,INTEGER,0,None,0


('comments',)


,0,1,2,3,4,5
0,0,subreddit_id,TEXT,0,None,0
1,1,post_id,TEXT,0,None,0
2,2,parent_id,TEXT,0,None,0
3,3,comment_id,TEXT,0,None,0
4,4,created_at,TIMESTAMP,0,None,0
5,5,last_modified_at,TIMESTAMP,0,None,0
6,6,score,INTEGER,0,None,0
7,7,upvote_ratio,REAL,0,None,0
8,8,gilded,INTEGER,0,None,0


('postlinks',)


...

,0,1,2,3,4,5
0,0,lang,TEXT,0,None,0
1,1,canonical_title,TEXT,0,None,0
2,2,other_title,TEXT,0,None,0


('revisions',)


,0,1,2,3,4,5
0,0,lang,TEXT,0,None,0
1,1,canonical_title,TEXT,0,None,0
2,2,revid,INTEGER,0,None,0
3,3,parentid,INTEGER,0,None,0
4,4,timestamp,TEXT,0,None,0


('pageviews',)


,0,1,2,3,4,5
0,0,lang,TEXT,0,None,0
1,1,title,TEXT,0,None,0
2,2,date,TIMESTAMP,0,None,0
3,3,pageviews,INTEGER,0,None,0


Rename cols

In [3]:
rename_dict = {'posts': {'id': 'post_id'},
               'comments': {'id': 'comment_id'},
               'postlinks': {'id': 'post_id'},
               'commentlinks': {'id': 'comment_id'},
               'resolved_redirects': {'redirected_title': 'canonical_title'},
               'revisions': {'title': 'canonical_title'}}

for table, columns in rename_dict.items():
    for old_name, new_name in columns.items():
        if old_name in pd.read_sql(f"SELECT * FROM {table} LIMIT 1", conn).columns:
            cursor.execute(f"ALTER TABLE {table} RENAME COLUMN {old_name} TO {new_name}")
conn.commit()

In [4]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

for table in cursor.fetchall():
    cursor.execute(f"PRAGMA table_info({table[0]})")
    print('\n', table)
    print(pd.DataFrame(cursor.fetchall())[[1, 2]])


 ('posts',)
                     1          2
0         subreddit_id       TEXT
1  crosspost_parent_id       TEXT
2              post_id       TEXT
3           created_at  TIMESTAMP
4           updated_at  TIMESTAMP
5        language_code       TEXT
6                score    INTEGER
7         upvote_ratio       REAL
8             gildings    INTEGER
9         num_comments    INTEGER

 ('comments',)
                  1          2
0      subreddit_id       TEXT
1           post_id       TEXT
2         parent_id       TEXT
3        comment_id       TEXT
4        created_at  TIMESTAMP
5  last_modified_at  TIMESTAMP
6             score    INTEGER
7      upvote_ratio       REAL
8            gilded    INTEGER

 ('postlinks',)
                     1        2
0              post_id     TEXT
1  end_processed_valid  INTEGER
2    end_processed_url     TEXT
3          final_valid  INTEGER
4         final_status  INTEGER
5            final_url     TEXT
6             in_title  INTEGER
7           re

In [15]:
# create index columns in pageviews and revisions

cursor.execute("CREATE INDEX IF NOT EXISTS pageviews_index ON pageviews (lang, title, date)")
cursor.execute("CREATE INDEX IF NOT EXISTS revisions_index ON revisions (lang, canonical_title, revid)")

Hash IDs

In [3]:
posts = pd.read_sql("SELECT * FROM posts", conn)
comments = pd.read_sql("SELECT * FROM comments", conn)
postlinks = pd.read_sql("SELECT * FROM postlinks", conn)
commentlinks = pd.read_sql("SELECT * FROM commentlinks", conn)



In [ ]:

def anonymise_key(key):
    if key:
        return hashlib.sha256(key.encode()).hexdigest()
    else:
        return key

for c in ['subreddit_id', 'crosspost_parent_id', 'post_id']:
    posts[c] = posts[c].apply(anonymise_key)

for c in ['subreddit_id', 'post_id', 'parent_id', 'comment_id']:
    comments[c] = comments[c].apply(anonymise_key)

for c in ['post_id']:
    postlinks[c] = postlinks[c].apply(anonymise_key)

for c in ['comment_id']:
    commentlinks[c] = commentlinks[c].apply(anonymise_key)

In [14]:
#  save to sql

posts.to_sql('posts', conn, if_exists='replace', index=False)
comments.to_sql('comments', conn, if_exists='replace', index=False)
postlinks.to_sql('postlinks', conn, if_exists='replace', index=False)
commentlinks.to_sql('commentlinks', conn, if_exists='replace', index=False)


11573367

# Get sample dataset

In [16]:
original_conn = sqlite3.connect("wikireddit.db")
new_conn = sqlite3.connect("wikireddit_sample.db")

# Sample 100 posts
posts_sample = pd.read_sql_query(
    """
    SELECT *
    FROM posts
    ORDER BY RANDOM()
    LIMIT 100
    """,
    original_conn
)

# Sample 500 comments
comments_sample = pd.read_sql_query(
    """
    SELECT *
    FROM comments
    ORDER BY RANDOM()
    LIMIT 500
    """,
    original_conn
)

# Extract sets or tuples of IDs for later filtering
sample_post_ids = tuple(posts_sample["post_id"].tolist())
sample_comment_ids = tuple(comments_sample["comment_id"].tolist())


In [17]:
posts_sample.to_sql("posts", new_conn, if_exists="replace", index=False)
comments_sample.to_sql("comments", new_conn, if_exists="replace", index=False)

500

In [18]:
# Copy postlinks for the 100 sampled posts
placeholder_postids = ",".join(["?"] * len(sample_post_ids))
query_copy_postlinks = f"""
    SELECT *
    FROM postlinks
    WHERE post_id IN ({placeholder_postids})
"""
# We'll read from the old DB, insert into new DB
postlinks_df = pd.read_sql_query(query_copy_postlinks, original_conn, params=sample_post_ids)
postlinks_df.to_sql("postlinks", new_conn, if_exists="replace", index=False)

# Copy commentlinks for the 1000 sampled comments
placeholder_commentids = ",".join(["?"] * len(sample_comment_ids))
query_copy_commentlinks = f"""
    SELECT *
    FROM commentlinks
    WHERE comment_id IN ({placeholder_commentids})
"""
commentlinks_df = pd.read_sql_query(query_copy_commentlinks, original_conn, params=sample_comment_ids)
commentlinks_df.to_sql("commentlinks", new_conn, if_exists="replace", index=False)


622

In [19]:
all_final_urls = set(postlinks_df["final_url"].dropna()) | set(commentlinks_df["final_url"].dropna())
all_final_urls = list(all_final_urls)  # convert set to list

placeholder_urls = ",".join(["?"] * len(all_final_urls))
query_copy_linkarticles = f"""
    SELECT *
    FROM linkarticles
    WHERE final_url IN ({placeholder_urls})
"""
linkarticles_df = pd.read_sql_query(
    query_copy_linkarticles, original_conn, params=all_final_urls
)
linkarticles_df.to_sql("linkarticles", new_conn, if_exists="replace", index=False)


750

In [20]:
resolved_redirects = pd.read_sql("SELECT * FROM resolved_redirects", con=original_conn)
collected_redirects = pd.read_sql("SELECT * FROM collected_redirects", con=original_conn)
wiki_ids = pd.read_sql("SELECT * FROM wiki_ids", con=original_conn)

resolved_redirects_sample = linkarticles_df.merge(resolved_redirects, on=['lang', 'raw_title'], how='left')[resolved_redirects.columns]
collected_redirects_sample = resolved_redirects_sample.merge(collected_redirects, on=['lang', 'canonical_title'], how='left')
wiki_ids_sample = resolved_redirects_sample.merge(wiki_ids, left_on=['lang', 'canonical_title'],
                                right_on=['lang', 'title'], how='left')[wiki_ids.columns]

# Pull from original, insert into new
resolved_redirects_sample.to_sql("resolved_redirects", new_conn, if_exists="replace", index=False)
collected_redirects_sample.to_sql("collected_redirects", new_conn, if_exists="replace", index=False)
wiki_ids_sample.to_sql('wiki_ids', con=new_conn, if_exists='replace', index=False)

750

In [21]:
lang_canon = resolved_redirects_sample[["lang", "canonical_title"]].dropna().drop_duplicates()

# Convert to list of tuples, e.g. [('en', 'Apple'), ('fr', 'État')]
lang_canon_tuples = list(lang_canon.itertuples(index=False, name=None))

print("Number of (lang, canonical_title) pairs:", len(lang_canon_tuples))


Number of (lang, canonical_title) pairs: 737


In [22]:

# Step A: Make placeholders like: "(?, ?), (?, ?), ..."
placeholders = ", ".join(["(?, ?)"] * len(lang_canon_tuples))

# Step B: Flatten the pairs for parameter substitution
flattened_params = []
for (l, c) in lang_canon_tuples:
    flattened_params.extend([l, c])

# Step C: Build and execute the query
query_pageviews = f"""
SELECT *
FROM pageviews
WHERE (lang, title) IN ({placeholders});
"""
pageviews_df = pd.read_sql_query(query_pageviews, original_conn, params=flattened_params)

# Step D: Insert into the new DB
pageviews_df.to_sql("pageviews", new_conn, if_exists="append", index=False)



600946

In [23]:
placeholders = ", ".join(["(?, ?)"] * len(lang_canon_tuples))
flattened_params = []
for (l, c) in lang_canon_tuples:
    flattened_params.extend([l, c])

query_revisions = f"""
SELECT *
FROM revisions
WHERE (lang, canonical_title) IN ({placeholders});
"""
revisions_df = pd.read_sql_query(query_revisions, original_conn, params=flattened_params)

revisions_df.to_sql("revisions", new_conn, if_exists="append", index=False)


172705

In [24]:
import os
mb_size = os.path.getsize('wikireddit_sample.db') / 1024 / 1024
print(f"Database size: {mb_size:.2f} MB")

Database size: 41.91 MB
